In [1]:
import sqlite3
netflix_conn = sqlite3.connect('netflix.sqlite')
task_conn = sqlite3.connect("task1.sqlite")

In [2]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d
netflix_conn.row_factory = dict_factory

In [3]:
task_cur = task_conn.cursor()
with task_conn:
  task_cur.execute("""
  CREATE TABLE films (
    film_id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT NOT NULL,
    title TEXT NOT NULL,
    director TEXT NOT NULL,
    country TEXT NOT NULL,
    date_added TEXT NOT NULL,
    release_year INTEGER NOT NULL,
    rating TEXT NOT NULL,
    duration TEXT NOT NULL,
    listed_in TEXT NOT NULL,
    description TEXT NOT NULL
  );
  """)
  task_cur.execute("""
  CREATE TABLE actors (
    actor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    actor_name TEXT UNIQUE NOT NULL
  );
  """)
  task_cur.execute("""
    CREATE TABLE actors_films (
    film_id INTEGER,
    actor_id INTEGER,
    PRIMARY KEY (actor_id, film_id),
    FOREIGN KEY (film_id) REFERENCES films(film_id),
    FOREIGN KEY (actor_id) REFERENCES actors(actor_id)
  );
  """)

In [4]:
netflix_cur = netflix_conn.cursor()
with netflix_conn:
  with task_conn:
    for row in netflix_cur.execute("SELECT * FROM netflix_titles"):
      task_cur.execute("""
      INSERT INTO films (type, title, director, country, date_added, release_year, rating, duration, listed_in, description) VALUES
        (:type, :title, :director, :country, :date_added, :release_year, :rating, :duration, :listed_in, :description);
       """, row)
      film_id = task_cur.execute("SELECT last_insert_rowid() FROM films;").fetchone()[0]
      for actor in row["cast"].split(", "):
        task_cur.execute("INSERT OR IGNORE INTO actors (actor_name) VALUES (?)", (actor,))
        task_cur.execute("""
        INSERT OR IGNORE INTO actors_films (film_id, actor_id) VALUES
        (?, (SELECT actor_id FROM actors WHERE actor_name = ?))""", (film_id, actor))

In [5]:
with task_conn:
  most_freq_pair = task_cur.execute("""
  SELECT (SELECT actor_name from actors where actor_id = id1) as name1,
  (SELECT actor_name from actors WHERE actor_id = id2) as name2
    FROM (select id1, id2, COUNT(*) FROM
      (SELECT A.actor_id as id1, B.actor_id as id2 FROM actors_films A, actors_films B
        WHERE A.film_id = B.film_id AND id1 < id2) T
        GROUP BY id1, id2
        ORDER BY COUNT(*) DESC
        LIMIT 1);
  """).fetchone()
most_freq_pair

('John Paul Tremblay', 'Robb Wells')